In [17]:
import polars as pl
import os
from datetime import datetime, timezone, timedelta
import csv

from sites_deloyment_times import deployment_times, datetime_format

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

In [18]:
sites_meta = pl.read_csv("sites.csv", separator=";").with_columns(pl.exclude(pl.Utf8).cast(str))

df_L1_1_min = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_L1_1_min_acropolis.parquet"))
df_L2_1_min = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_L2_1_min_acropolis.parquet"))
df_L2_1_h = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_L2_1_h_acropolis.parquet"))

In [13]:
print(deployment_times["TUMR"])
sites_meta.filter(pl.col("site") == "TUMR")

{'sensors': [{'id': 4, 'start_time': '2024-02-14T00:00:00+0000', 'end_time': '2024-7-1T00:00:00+0000'}, {'id': 6, 'start_time': '2024-7-1T00:00:00+0000', 'end_time': '2024-12-16T09:09:28+0000'}]}


site,site_name,latitude,longitude,elevation,site_type,installation_classification,height_of_building,responsible_party
str,str,str,str,str,str,str,str,str
"""TUMR""","""TUM Zentralgelände Nord Maxvor…","""48.150733""","""11.569168""","""511.39""","""city""","""rooftop""","""31.06""","""tum:environmental_sensing and_…"


In [14]:
# Utility

def extract_site_data(df, deployment_times: dict, site_name:str):
    extracted_dates = []
    
    for sensor in deployment_times[site_name]["sensors"]:
        
        id = sensor["id"]
        start_time = datetime.strptime(sensor["start_time"], datetime_format)
        end_time = datetime.strptime(sensor["end_time"], datetime_format)       
        
        df_temp = df.filter(pl.col("system_id") == id)  \
            .filter(pl.col("creation_timestamp") \
            .is_between(start_time, end_time)) \
            .collect()
        
        extracted_dates.append(df_temp)
        
    return pl.concat(extracted_dates)
  
def calculate_decimal_year(date) -> float:
    year = date.year 
    today = (date - datetime(year, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc)).total_seconds()
    seconds_total_year = (datetime(year, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc) - datetime(year-1, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc)).total_seconds()

    x=  ((today / seconds_total_year) + year)
    return float("{:.6f}".format(x))

# 1 min data (Level 1)

In [15]:
df_L1_1_min.select(pl.col("Flag")).unique().collect()

Flag
str
"""U"""
"""H"""


In [19]:
for site in deployment_times.keys():
    print(site)
    
    df_temp = extract_site_data(df=df_L1_1_min, deployment_times = deployment_times, site_name=site) \
        .fill_null('') \
        .rename({"gmp343_corrected": "co2", "h2o_v%": "h2o", "enclosure_bme280_pressure":"pressure", "gmp343_temperature":"sensor_temperature", "wxt532_speed_avg": "ws", "wxt532_direction_avg":"wd"}) \
        .with_columns((pl.col("creation_timestamp") + timedelta(seconds=30))) \
        .with_columns(
            pl.col("co2").round(2),
            pl.col("h2o").round(2),
            pl.col("pressure").round(2),
            pl.col("sensor_temperature").round(2),
            pl.col("ws").round(2),
            pl.col("wd").round(2)) \
        .with_columns(
            (pl.col("creation_timestamp").dt.year()).alias("Year"),
            (pl.col("creation_timestamp").dt.month()).alias("Month"),
            (pl.col("creation_timestamp").dt.day()).alias("Day"),
            (pl.col("creation_timestamp").dt.hour()).alias("Hour"),
            (pl.col("creation_timestamp").dt.minute()).alias("Minute"),
            (pl.col("creation_timestamp").dt.second()).alias("Second"),
            (pl.col('creation_timestamp').dt.to_string("%Y-%m-%d %H:%M:%S")).alias("#Datetime")) \
        .with_columns(pl.struct(['creation_timestamp']) \
        .map_elements(lambda x: calculate_decimal_year(x['creation_timestamp']), return_dtype=pl.Float64) \
        .alias("DecimalDate")) \
        .select(["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag"]) \
        .with_columns(pl.exclude(pl.Utf8).cast(str)) \
        .fill_null('')
        
    # construct icos cities portal head

    data_level = 1
    header_lines = 45
    file_name = f"{site}_munich_acropolis_L{data_level}_1min.csv"
    file_lines = len(df_temp) + header_lines
    site_short_name = site[:4]
    site_long_name = sites_meta.filter(pl.col("site") == site[:4]).select("site_name").item()
    latitude = sites_meta.filter(pl.col("site") == site[:4]).select("latitude").item()
    longitude = sites_meta.filter(pl.col("site") == site[:4]).select("longitude").item()
    altitude = sites_meta.filter(pl.col("site") == site[:4]).select("elevation").item()
    sampling_height = sites_meta.filter(pl.col("site") == site[:4]).select("height_of_building").item()
    start_date = df_temp.select("#Datetime").row(0)[0]
    stop_date = df_temp.select("#Datetime").row(-1)[0]

    with open(os.path.join(DATA_DIRECTORY,"processed", "icos_cities_portal_processing", f"level_{data_level}", file_name), 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';', lineterminator='\n')
        field = ["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag"]
    
        writer.writerow(["# TITLE: co2 - continuous time series from low and mid cost sensors"])
        writer.writerow([f"# FILE NAME: {file_name}"])
        writer.writerow(["# DATA FORMAT: see the last line of this header for column description"])
        writer.writerow([f'# TOTAL LINES: {file_lines}'])
        writer.writerow([f'# HEADER LINES: {header_lines}'])
        writer.writerow(['# PROJECT: ICOS CITIES'])
        writer.writerow([f'# DATA VERSION: L{data_level}'])
        writer.writerow([f'# STATION CODE: {site_short_name}'])
        writer.writerow([f'# STATION NAME: {site_long_name} ({site_short_name})'])
        writer.writerow(['# OBSERVATION CATEGORY: Air sampling observation at a stationary platform'])
        writer.writerow(['# COUNTRY/TERRITORY: DE'])
        writer.writerow([r'# RESPONSIBLE INSTITUTE: TUM, Technial University Munich'])
        writer.writerow(['# CONTRIBUTOR:  Patrick Aigner, Jia Chen, Klaus Kürzinger'])
        writer.writerow(['# CONTACT POINT: Patrick Aigner <patrick.aigner@tum.de>, Jia Chen <jia.chen@tum.de>'])
        writer.writerow(["# FUNDING: European Union's Horizon 2020 Research and Innovation Programme, Grant Agreement No. 101037319"])
        writer.writerow([f'# LATITUDE: {latitude}'])
        writer.writerow([f'# LONGITUDE: {longitude}'])
        writer.writerow([f'# ALTITUDE: {altitude} m asl'])
        
        if site[:4] == "BLUT":
            writer.writerow([f'# SAMPLING HEIGHTS: {site[-2:]} m agl'])
        else:
            writer.writerow([f'# SAMPLING HEIGHTS: {sampling_height} m agl'])
        
        writer.writerow(['# PARAMETER: co2'])
        writer.writerow([f'# COVERING PERIOD: {start_date} - {stop_date}'])
        writer.writerow(['# TIME INTERVAL: 1 minute'])
        writer.writerow(['# MEASUREMENT UNIT: µmol/mol'])
        writer.writerow(['# MEASUREMENT METHOD: NDIR'])
        writer.writerow(['# INSTRUMENT: Vaisala GMP343'])
        writer.writerow(['# SAMPLING TYPE: continuous'])
        writer.writerow(['# TIME ZONE: Central European Time (UTC+1), Central European Summer Time (UTC+2)'])
        writer.writerow(['# MEASUREMENT SCALE: WMO-CO2-X2019'])
        writer.writerow(['# DATA POLICY: ICOS CITIES DATA is licensed under a Creative Commons Attribution 4.0 international licence (http://creativecommons.org/licenses/by/4.0/.The ICOS CITIES data licence is described at https://data.icos-cities.eu/licence.'])
        writer.writerow(['# COMMENT:'])
        writer.writerow(['#'])
        writer.writerow(['#   - Times are UTC+0'])
        writer.writerow(['#   - Time-averaged values are reported at the middle of the averaging interval.'])
        writer.writerow(['#   - co2: dry mole air fraction (µmol/mol)'])
        writer.writerow(['#   - h20: absolute humidity in vol%'])
        writer.writerow(['#   - pressure: ambient pressure of outdoor enclosure in hPa'])
        writer.writerow(['#   - sensor_temperature: measurement chamber temperature in °C'])
        writer.writerow(['#   - ws: wind speed in m/s'])
        writer.writerow(['#   - wd: wind direction in degrees'])
        writer.writerow(["#   - Flag 'U' = data correct before manual quality control"])
        writer.writerow(["#   - Flag 'H' = Potentially locally contaminated by hampel filter (auto)"])
        writer.writerow(['#   - In case of gaps between instruments, the timeseries are filled with empty string'])
        writer.writerow(['#   - Release notes: '])
        writer.writerow(['#'])
        writer.writerow(field)
        
        for row in df_temp.iter_rows():
            writer.writerow([''.join(item) for item in row])


TUMR
FELR
TAUR
DLRR
SENR
RDIR
SCHR
FINR
SWMR
MAIR
PASR
GROR
BLUT_48
BLUT_85
NPLR
BOGR
HARR
BALR


# 1 minute data (Level 2)

In [7]:
df_L2_1_min.select(pl.col("Flag")).unique().collect()

Flag
str
"""H"""
"""C"""
"""O"""


In [8]:
for site in deployment_times.keys():
    print(site)
    
    df_temp = extract_site_data(df=df_L2_1_min, deployment_times = deployment_times, site_name=site) \
        .fill_null('') \
        .rename({"gmp343_corrected": "co2", "h2o_v%": "h2o", "enclosure_bme280_pressure":"pressure", "gmp343_temperature":"sensor_temperature", "wxt532_speed_avg": "ws", "wxt532_direction_avg":"wd"}) \
        .with_columns((pl.col("creation_timestamp") + timedelta(seconds=30))) \
        .with_columns(
            pl.col("co2").round(2),
            pl.col("h2o").round(2),
            pl.col("pressure").round(2),
            pl.col("sensor_temperature").round(2),
            pl.col("ws").round(2),
            pl.col("wd").round(2)) \
        .with_columns(
            (pl.col("creation_timestamp").dt.year()).alias("Year"),
            (pl.col("creation_timestamp").dt.month()).alias("Month"),
            (pl.col("creation_timestamp").dt.day()).alias("Day"),
            (pl.col("creation_timestamp").dt.hour()).alias("Hour"),
            (pl.col("creation_timestamp").dt.minute()).alias("Minute"),
            (pl.col("creation_timestamp").dt.second()).alias("Second"),
            (pl.col('creation_timestamp').dt.to_string("%Y-%m-%d %H:%M:%S")).alias("#Datetime")) \
        .with_columns(pl.struct(['creation_timestamp']) \
        .map_elements(lambda x: calculate_decimal_year(x['creation_timestamp']), return_dtype=pl.Float64) \
        .alias("DecimalDate")) \
        .select(["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag"]) \
        .with_columns(pl.exclude(pl.Utf8).cast(str)) \
        .fill_null('')
        
    # construct icos cities portal head

    header_lines = 46
    data_level = 2
    file_name = f"{site}_munich_acropolis_L{data_level}_1min.csv"
    file_lines = len(df_temp) + header_lines
    site_short_name = site[:4]
    site_long_name = sites_meta.filter(pl.col("site") == site[:4]).select("site_name").item()
    latitude = sites_meta.filter(pl.col("site") == site[:4]).select("latitude").item()
    longitude = sites_meta.filter(pl.col("site") == site[:4]).select("longitude").item()
    altitude = sites_meta.filter(pl.col("site") == site[:4]).select("elevation").item()
    sampling_height = sites_meta.filter(pl.col("site") == site[:4]).select("height_of_building").item()
    start_date = df_temp.select("#Datetime").row(0)[0]
    stop_date = df_temp.select("#Datetime").row(-1)[0]

    with open(os.path.join(DATA_DIRECTORY,"processed", "icos_cities_portal_processing", f"level_{data_level}", file_name), 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';', lineterminator='\n')
        field = ["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag"]
    
        writer.writerow(["# TITLE: co2 - continuous time series from low and mid cost sensors"])
        writer.writerow([f"# FILE NAME: {file_name}"])
        writer.writerow(["# DATA FORMAT: see the last line of this header for column description"])
        writer.writerow([f'# TOTAL LINES: {file_lines}'])
        writer.writerow([f'# HEADER LINES: {header_lines}'])
        writer.writerow(['# PROJECT: ICOS CITIES'])
        writer.writerow([f'# DATA VERSION: L{data_level}'])
        writer.writerow([f'# STATION CODE: {site_short_name}'])
        writer.writerow([f'# STATION NAME: {site_long_name} ({site_short_name})'])
        writer.writerow(['# OBSERVATION CATEGORY: Air sampling observation at a stationary platform'])
        writer.writerow(['# COUNTRY/TERRITORY: DE'])
        writer.writerow([r'# RESPONSIBLE INSTITUTE: TUM, Technial University Munich'])
        writer.writerow(['# CONTRIBUTOR:  Patrick Aigner, Jia Chen, Klaus Kürzinger'])
        writer.writerow(['# CONTACT POINT: Patrick Aigner <patrick.aigner@tum.de>, Jia Chen <jia.chen@tum.de>'])
        writer.writerow(["# FUNDING: European Union's Horizon 2020 Research and Innovation Programme, Grant Agreement No. 101037319"])
        writer.writerow([f'# LATITUDE: {latitude}'])
        writer.writerow([f'# LONGITUDE: {longitude}'])
        writer.writerow([f'# ALTITUDE: {altitude} m asl'])
        
        if site[:4] == "BLUT":
            writer.writerow([f'# SAMPLING HEIGHTS: {site[-2:]} m agl'])
        else:
            writer.writerow([f'# SAMPLING HEIGHTS: {sampling_height} m agl'])
        
        writer.writerow(['# PARAMETER: co2'])
        writer.writerow([f'# COVERING PERIOD: {start_date} - {stop_date}'])
        writer.writerow(['# TIME INTERVAL: 1 minute'])
        writer.writerow(['# MEASUREMENT UNIT: µmol/mol'])
        writer.writerow(['# MEASUREMENT METHOD: NDIR'])
        writer.writerow(['# INSTRUMENT: Vaisala GMP343'])
        writer.writerow(['# SAMPLING TYPE: continuous'])
        writer.writerow(['# TIME ZONE: Central European Time (UTC+1), Central European Summer Time (UTC+2)'])
        writer.writerow(['# MEASUREMENT SCALE: WMO-CO2-X2019'])
        writer.writerow(['# DATA POLICY: ICOS CITIES DATA is licensed under a Creative Commons Attribution 4.0 international licence (http://creativecommons.org/licenses/by/4.0/.The ICOS CITIES data licence is described at https://data.icos-cities.eu/licence.'])
        writer.writerow(['# COMMENT:'])
        writer.writerow(['#'])
        writer.writerow(['#   - Times are UTC+0'])
        writer.writerow(['#   - Time-averaged values are reported at the middle of the averaging interval.'])
        writer.writerow(['#   - co2: dry mole air fraction (µmol/mol)'])
        writer.writerow(['#   - h20: absolute humidity in vol%'])
        writer.writerow(['#   - pressure: ambient pressure of outdoor enclosure in hPa'])
        writer.writerow(['#   - sensor_temperature: measurement chamber temperature in °C'])
        writer.writerow(['#   - ws: wind speed in m/s'])
        writer.writerow(['#   - wd: wind direction in degrees'])
        writer.writerow(["#   - Flag 'O' = data correct after manual quality control"])
        writer.writerow(["#   - Flag 'H' = Potentially locally contaminated by hampel filter (auto)"])
        writer.writerow(["#   - Flag 'C' = Potentially locally contaminated (manual quality control)"])
        writer.writerow(['#   - In case of gaps between instruments, the timeseries are filled with empty string'])
        writer.writerow(['#   - Release notes: '])
        writer.writerow(['#'])
        writer.writerow(field)
        
        for row in df_temp.iter_rows():
            writer.writerow([''.join(item) for item in row])


TUMR
FELR
TAUR
DLRR
SENR
RDIR
SCHR
FINR
SWMR
MAIR
PASR
GROR
BLUT_48
BLUT_85
NPLR
BOGR
HARR
BALR


# 1 hour data (Level 2)

In [9]:
df_L2_1_h.select(pl.col("Flag")).unique().collect()

Flag
str
"""O"""
"""K"""


In [10]:
for site in deployment_times.keys():
    print(site)
    
    df_temp = extract_site_data(df=df_L2_1_h, deployment_times = deployment_times, site_name=site) \
        .fill_null('') \
        .rename({"gmp343_corrected": "co2", "h2o_v%": "h2o", "enclosure_bme280_pressure":"pressure", "gmp343_temperature":"sensor_temperature", "wxt532_speed_avg": "ws", "wxt532_direction_avg":"wd"}) \
        .with_columns(
            pl.col("co2").round(2),
            pl.col("h2o").round(2),
            pl.col("pressure").round(2),
            pl.col("sensor_temperature").round(2),
            pl.col("ws").round(2),
            pl.col("wd").round(2),
            pl.col("Stdev").round(2)) \
        .with_columns(
            (pl.col("creation_timestamp").dt.year()).alias("Year"),
            (pl.col("creation_timestamp").dt.month()).alias("Month"),
            (pl.col("creation_timestamp").dt.day()).alias("Day"),
            (pl.col("creation_timestamp").dt.hour()).alias("Hour"),
            (pl.col("creation_timestamp").dt.minute()).alias("Minute"),
            (pl.col("creation_timestamp").dt.second()).alias("Second"),
            (pl.col('creation_timestamp').dt.to_string("%Y-%m-%d %H:%M:%S")).alias("#Datetime")) \
        .with_columns(pl.struct(['creation_timestamp']) \
        .map_elements(lambda x: calculate_decimal_year(x['creation_timestamp']), return_dtype=pl.Float64) \
        .alias("DecimalDate")) \
        .select(["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "NbPoints", "Stdev", "Flag"]) \
        .with_columns(pl.exclude(pl.Utf8).cast(str)) \
        .fill_null('')
        
    # construct icos cities portal head

    header_lines = 45
    data_level = 2
    file_name = f"{site}_munich_acropolis_L{data_level}_1h.csv"
    file_lines = len(df_temp) + header_lines         
    site_short_name = site[:4]
    site_long_name = sites_meta.filter(pl.col("site") == site[:4]).select("site_name").item()
    latitude = sites_meta.filter(pl.col("site") == site[:4]).select("latitude").item()
    longitude = sites_meta.filter(pl.col("site") == site[:4]).select("longitude").item()
    altitude = sites_meta.filter(pl.col("site") == site[:4]).select("elevation").item()
    sampling_height = sites_meta.filter(pl.col("site") == site[:4]).select("height_of_building").item()
    start_date = df_temp.select("#Datetime").row(0)[0]
    stop_date = df_temp.select("#Datetime").row(-1)[0]

    with open(os.path.join(DATA_DIRECTORY,"processed", "icos_cities_portal_processing", f"level_{data_level}", file_name), 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';', lineterminator='\n')
        field = ["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "NbPoints", "Stdev", "Flag"]
    
        writer.writerow(["# TITLE: co2 - continuous time series from low and mid cost sensors"])
        writer.writerow([f"# FILE NAME: {file_name}"])
        writer.writerow(["# DATA FORMAT: see the last line of this header for column description"])
        writer.writerow([f'# TOTAL LINES: {file_lines}'])
        writer.writerow([f'# HEADER LINES: {header_lines}'])
        writer.writerow(['# PROJECT: ICOS CITIES'])
        writer.writerow([f'# DATA VERSION: L{data_level}'])
        writer.writerow([f'# STATION CODE: {site_short_name}'])
        writer.writerow([f'# STATION NAME: {site_long_name} ({site_short_name})'])
        writer.writerow(['# OBSERVATION CATEGORY: Air sampling observation at a stationary platform'])
        writer.writerow(['# COUNTRY/TERRITORY: DE'])
        writer.writerow([r'# RESPONSIBLE INSTITUTE: TUM, Technial University Munich'])
        writer.writerow(['# CONTRIBUTOR:  Patrick Aigner, Jia Chen, Klaus Kürzinger'])
        writer.writerow(['# CONTACT POINT: Patrick Aigner <patrick.aigner@tum.de>, Jia Chen <jia.chen@tum.de>'])
        writer.writerow(["# FUNDING: European Union's Horizon 2020 Research and Innovation Programme, Grant Agreement No. 101037319"])
        writer.writerow([f'# LATITUDE: {latitude}'])
        writer.writerow([f'# LONGITUDE: {longitude}'])
        writer.writerow([f'# ALTITUDE: {altitude} m asl'])
        
        if site[:4] == "BLUT":
            writer.writerow([f'# SAMPLING HEIGHTS: {site[-2:]} m agl'])
        else:
            writer.writerow([f'# SAMPLING HEIGHTS: {sampling_height} m agl'])
        
        writer.writerow(['# PARAMETER: co2'])
        writer.writerow([f'# COVERING PERIOD: {start_date} - {stop_date}'])
        writer.writerow(['# TIME INTERVAL: hourly'])
        writer.writerow(['# MEASUREMENT UNIT: µmol/mol'])
        writer.writerow(['# MEASUREMENT METHOD: NDIR'])
        writer.writerow(['# INSTRUMENT: Vaisala GMP343'])
        writer.writerow(['# SAMPLING TYPE: continuous'])
        writer.writerow(['# TIME ZONE: Central European Time (UTC+1), Central European Summer Time (UTC+2)'])
        writer.writerow(['# MEASUREMENT SCALE: WMO-CO2-X2019'])
        writer.writerow(['# DATA POLICY: ICOS CITIES DATA is licensed under a Creative Commons Attribution 4.0 international licence (http://creativecommons.org/licenses/by/4.0/.The ICOS CITIES data licence is described at https://data.icos-cities.eu/licence.'])
        writer.writerow(['# COMMENT:'])
        writer.writerow(['#'])
        writer.writerow(['#   - Times are UTC+0'])
        writer.writerow(['#   - Time-averaged values are reported at the middle of the averaging interval.'])
        writer.writerow(['#   - co2: dry mole air fraction (µmol/mol)'])
        writer.writerow(['#   - h20: absolute humdity in vol%'])
        writer.writerow(['#   - pressure: ambient pressure of outdoor enclosure in hPa'])
        writer.writerow(['#   - sensor_temperature: measurement chamber temperature in °C'])
        writer.writerow(['#   - ws: wind speed in m/s'])
        writer.writerow(['#   - wd: wind direction in degrees'])
        writer.writerow(["#   - Flag 'O' = data correct after manual quality control"])
        writer.writerow(["#   - Flag 'K' = data incorrect after manual quality control"])
        writer.writerow(['#   - In case of gaps between instruments, the timeseries are filled with empty string'])
        writer.writerow(['#   - Release notes: '])
        writer.writerow(['#'])
        writer.writerow(field)
        
        for row in df_temp.iter_rows():
            writer.writerow([''.join(item) for item in row])


TUMR
FELR
TAUR
DLRR
SENR
RDIR
SCHR
FINR
SWMR
MAIR
PASR
GROR
BLUT_48
BLUT_85
NPLR
BOGR
HARR
BALR
